<a href="https://colab.research.google.com/github/softtechi007/TaskForge/blob/main/TaskForge_%E2%80%93_Console_Based_Task_Manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TaskForge – Console-Based Task Manager

Class task

In [ ]:
import json
import os
import datetime

class Task:
    id_counter = 1  # Static counter to generate unique IDs

    def __init__(self, title, priority, due_date, status="Pending", task_id=None):
        self.id = task_id if task_id else Task.id_counter
        self.title = title
        self.priority = priority
        self.due_date = due_date
        self.status = status
        if not task_id:
            Task.id_counter += 1

    def to_dict(self):
        """Convert task to dictionary for saving."""
        return {
            "id": self.id,
            "title": self.title,
            "priority": self.priority,
            "due_date": self.due_date,
            "status": self.status
        }

    @classmethod
    def from_dict(cls, data):
        """Create a Task object from dictionary data."""
        return cls(
            title=data["title"],
            priority=data["priority"],
            due_date=data["due_date"],
            status=data["status"],
            task_id=data["id"]
        )

Class Task Manager

In [ ]:
class TaskManager:
  def __init__(self):
      self.task_list = []
      self.load_from_file()


  def add_task(self):
    PRIORITY_OPTIONS = ["Low", "Medium", "High"]

    try:
        # --- Title ---
        while True:
            title = input("Enter task title: ").strip()
            if title:
                break
            print("Title cannot be empty. Please try again.")

        # --- Priority ---
        while True:
            priority = input(f"Enter priority (Low/Medium/High): ").strip().capitalize()
            if priority in PRIORITY_OPTIONS:
                break
            print("Invalid priority. Please choose from: Low,Medium,High")

        # --- Due Date ---
        while True:
            due_date_input = input("Enter due date (YYYY-MM-DD): ")
            try:
                due_date_obj = datetime.date.fromisoformat(due_date_input)
                due_date = due_date_obj.isoformat()  # Ensures consistent format
                break
            except ValueError:
                print("Invalid date format. Please enter date in YYYY-MM-DD format.")

        # --- Create and Save Task ---
        task = Task(title, priority, due_date)
        self.task_list.append(task)
        print(" Task added successfully!")

    except Exception as e:
        print(f" Unexpected error task is not added: {e}")


  def update_task(self):
      if not self.task_list:
          print("No tasks available to update.")
          return

      task_id = input("Enter task ID to update: ")
      task = self.get_task_by_id(task_id)

      # --- Update Title ---
      new_title = input(f"New title (current: {task.title}): ").strip()
      if new_title:
          task.title = new_title

      # --- Update Priority ---
      PRIORITY_OPTIONS = ["Low", "Medium", "High"]
      new_priority = input(f"New priority (current: {task.priority}): ").strip().capitalize()
      if new_priority in PRIORITY_OPTIONS:
          task.priority = new_priority
      elif new_priority:
          print("Invalid priority. Keeping old value.")

      # --- Update Due Date ---
      new_due_date = input(f"New due date (current: {task.due_date}): ").strip()
      if new_due_date:
          try:
              datetime.date.fromisoformat(new_due_date)
              task.due_date = new_due_date
          except ValueError:
              print("Invalid date format. Keeping old value.")

      print("Task updated successfully!")


  def mark_complete(self):
      if not self.task_list:
          print("No tasks available to mark as complete.")
          return

      task_id = input("Enter task ID to Mark Complete: ")
      task = self.get_task_by_id(task_id)  # Automatically handles input + validation
      if task.status == "Completed":
          print("Task is already marked as completed.")
      else:
          task.status = "Completed"
          print("Task marked as completed.")


  def delete_task(self):
      if not self.task_list:
          print("No tasks available to delete.")
          return

      task_id = input("Enter task ID to Delete: ")
      task = self.get_task_by_id(task_id)
      if task:
          self.task_list.remove(task)
          print("Task deleted successfully.")
      else:
          print("Task not found.")


  def get_task_by_id(self, task_id=None):
    """Prompts for a valid task ID and returns the corresponding Task object."""
    while True:
          # If no task_id passed in, or if invalid, prompt user
          if task_id is None or not any(task.id == task_id for task in self.task_list):
              print("\nAvailable Tasks:")
              for task in self.task_list:
                  print(f"  ID: {task.id} | Title: {task.title} | Status: {task.status}")

              task_id_input = input("Please enter a valid Task ID: ").strip()

              if not task_id_input.isdigit():
                  print("Invalid input. Please enter a numeric Task ID.")
                  task_id = None  # Force loop
                  continue

              task_id = int(task_id_input)

          for task in self.task_list:
              if task.id == task_id:
                  return task

          print(f"Task with ID {task_id} not found. Please try again.")
          task_id = None  # Trigger another loop



  def view_tasks(self):
      """Displays tasks with optional filtering"""
      if not self.task_list:
          print("No tasks found.")
          return

      # Ask if user wants to filter
      while True:
        filter_choice = input("\nView all tasks (A) or apply filters (F)? [A/F]: ").strip().lower()
        if filter_choice == 'a':
            self._display_tasks(self.task_list)
            break
        elif filter_choice == 'f':
            filtered_tasks = self._get_filtered_tasks()
            self._display_tasks(filtered_tasks)
            break
        else:
            print("Invalid input. Please enter 'A' to view all tasks or 'F' to apply filters.")

  def _get_filtered_tasks(self):
      """Handles the filtering logic"""
      print("\nAvailable filters:")
      print("1. By Status")
      print("2. By Due Date")
      print("3. Both Status and Due Date")
      print("4. Cancel (show all)")

      while True:
          choice = input("Choose filter option (1-4): ").strip()

          if choice == '1':
              return self._filter_by_status()
          elif choice == '2':
              return self._filter_by_due_date()
          elif choice == '3':
              return self._filter_by_both()
          elif choice == '4':
              return self.task_list
          else:
              print("Invalid choice. Please try again.")

  def _filter_by_status(self):
      """Filters tasks by status"""
      while True:
          status = input("Enter status (Pending/Completed): ").strip().capitalize()
          if status in ['Pending', 'Completed']:
              return [task for task in self.task_list if task.status == status]
          print("Invalid status. Please enter 'Pending' or 'Completed'")

  def _get_week_boundaries(self):
      """Returns start (Monday) and end (Sunday) of current calendar week"""
      today = datetime.date.today()
      start_of_week = today - datetime.timedelta(days=today.weekday())  # Monday
      end_of_week = start_of_week + datetime.timedelta(days=6)         # Sunday
      return start_of_week, end_of_week

  def _filter_by_due_date(self):
      """Filters tasks by due date"""
      today = datetime.date.today()
      start_week, end_week = self._get_week_boundaries()

      print("\nDue date options:")
      print("1. Due Today")
      print(f"2. Due This Week ({start_week.isoformat()} to {end_week.isoformat()})")

      while True:
          choice = input("Choose due date filter (1-2): ").strip()

          if choice == '1':
              target_date = today.isoformat()
              return [task for task in self.task_list if task.due_date == target_date]
          elif choice == '2':
              print(f"\n--- Tasks Due This Week ({start_week.isoformat()} to {end_week.isoformat()}) ---")
              return [
                  task for task in self.task_list
                  if start_week <= datetime.date.fromisoformat(task.due_date) <= end_week
              ]
          else:
              print("Invalid choice. Please try again.")

  def _filter_by_both(self):
      """Filters by both status and due date"""
      status_tasks = self._filter_by_status()
      today = datetime.date.today()
      start_week, end_week = self._get_week_boundaries()

      print("\nNow select due date filter:")
      print("1. Due Today")
      print(f"2. Due This Week ({start_week.isoformat()} to {end_week.isoformat()})")

      while True:
          choice = input("Choose due date filter (1-2): ").strip()

          if choice == '1':
              target_date = today.isoformat()
              return [task for task in status_tasks if task.due_date == target_date]
          elif choice == '2':
              print(f"\n--- Tasks Due This Week ({start_week.isoformat()} to {end_week.isoformat()}) ---")
              return [
                  task for task in status_tasks
                  if start_week <= datetime.date.fromisoformat(task.due_date) <= end_week
              ]
          else:
              print("Invalid choice. Please try again.")

  def _display_tasks(self, tasks):
      """Displays tasks in a formatted table"""
      if not tasks:
          print("No tasks match the selected filters.")
          return

      print("\n--- Task List ---")
      print(f"{'ID':<8} {'Title':<25} {'Priority':<12} {'Due Date':<12} {'Status':<10}")
      print("-" * 70)
      for task in tasks:
          print(f"{task.id:<8} {task.title[:24]:<25} {task.priority:<12} {task.due_date:<12} {task.status:<10}")


  def save_to_file(self):
      data = [task.to_dict() for task in self.task_list]
      with open("tasks.json", "w") as f:
          json.dump(data, f, indent=4)
      print("Tasks saved to file.")


  def load_from_file(self):
      if os.path.exists("tasks.json"):
          with open("tasks.json", "r") as f:
              data = json.load(f)
              self.task_list = [Task.from_dict(item) for item in data]
              if self.task_list:
                  Task.id_counter = max(task.id for task in self.task_list) + 1
          print("Tasks loaded from file.")

Main function for running the code

In [ ]:
def main():
    manager = TaskManager()

    while True:
        print("\n--- Task Manager Menu ---")
        print("1. Add Task")
        print("2. View Tasks")
        print("3. Update Task")
        print("4. Mark Task as Complete")
        print("5. Delete Task")
        print("6. Save & Exit")

        choice = input("Enter your choice (1-6): ").strip()

        match choice:
            case "1":
                manager.add_task()
            case "2":
                manager.view_tasks()
            case "3":
                manager.update_task()
            case "4":
                manager.mark_complete()
            case "5":
                manager.delete_task()
            case "6":
                manager.save_to_file()
                break
            case _:
                print("Invalid choice. Try again.")

if __name__ == "__main__":
    main()

Tasks loaded from file.

--- Task Manager Menu ---
1. Add Task
2. View Tasks
3. Update Task
4. Mark Task as Complete
5. Delete Task
6. Save & Exit
Enter your choice (1-6): 2

View all tasks (A) or apply filters (F)? [A/F]: f

Available filters:
1. By Status
2. By Due Date
3. Both Status and Due Date
4. Cancel (show all)
Choose filter option (1-4): 2

Due date options:
1. Due Today
2. Due This Week (2025-06-30 to 2025-07-06)
Choose due date filter (1-2): 3
Invalid choice. Please try again.
Choose due date filter (1-2): 1

--- Task List ---
ID       Title                     Priority     Due Date     Status    
----------------------------------------------------------------------
3        Bannana                   High         2025-07-04   Pending   

--- Task Manager Menu ---
1. Add Task
2. View Tasks
3. Update Task
4. Mark Task as Complete
5. Delete Task
6. Save & Exit
Enter your choice (1-6): 2

View all tasks (A) or apply filters (F)? [A/F]: f

Available filters:
1. By Status
2. By 